# 1. Importaciones

In [1]:
import numpy as np
import pandas as pd
import random
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler

# 2. Generando Dataset

In [2]:
random.seed(222)
np.random.seed(222)

# Diccionario de valores
valor_plan = {
    'basico': 4.99,
    'estandar': 9.99,
    'premium': 14.99
}

data = []
n = 5000

for _ in range(n):
    plan = random.choices(['basico', 'estandar', 'premium'], weights=[0.3, 0.5, 0.2])[0]
    antiguedad = random.randint(1, 60)
    facturas_impagas = random.randint(0, 3)
    frecuencia_uso = random.randint(0, 30)
    metodo_pago = random.choice(['transferencia_bancaria', 'tarjeta_credito', 'tarjeta_debito', 'efectivo'])

    tickets_soporte = random.randint(0, 10)
    tipo_contrato = random.choices(['mensual', 'anual'], weights=[0.7, 0.3])[0]
    cambios_plan = random.randint(0, 3)
    canal_adquisicion = random.choice(['web', 'referido', 'redes_sociales', 'call_center'])

    # Probabilidad base de churn
    prob_churn = 0.030

    # Antiguedad
    if antiguedad < 6:
        prob_churn += 0.09
    elif antiguedad < 12:
        prob_churn += 0.05
    elif antiguedad > 36:
        prob_churn -= 0.10

    # Plan
    if plan == 'basico':
        prob_churn += 0.05
    elif plan == 'premium':
        prob_churn -= 0.08

    # Método de pago
    if metodo_pago == 'efectivo':
        prob_churn += 0.10
    elif metodo_pago == 'transferencia_bancaria':
        prob_churn += 0.05
    else:
        prob_churn -= 0.08

    # Facturas impagas
    if facturas_impagas == 0:
        prob_churn -= 0.18
    elif facturas_impagas == 1:
        prob_churn += 0.07
    elif facturas_impagas >= 2:
        prob_churn += 0.16

    # Frecuencia de uso
    if frecuencia_uso < 5:
        prob_churn += 0.10
    elif frecuencia_uso < 10:
        prob_churn += 0.05
    elif frecuencia_uso > 20:
        prob_churn -= 0.10

    # Tickets de soporte
    if tickets_soporte == 0:
        prob_churn -= 0.06
    elif tickets_soporte > 5:
        prob_churn += 0.12
    elif tickets_soporte > 2:
        prob_churn += 0.06

    # Tipo de contrato
    if tipo_contrato == 'anual':
        prob_churn -= 0.10
    else:
        prob_churn += 0.10

    # Cambios de plan
    if cambios_plan >= 2:
        prob_churn += 0.12
    elif cambios_plan == 1:
        prob_churn += 0.06

    # Canal de adquisición
    if canal_adquisicion == 'referido':
        prob_churn -= 0.08
    elif canal_adquisicion == 'call_center':
        prob_churn += 0.06

    prob_churn = min(max(prob_churn, 0.01), 0.95)

    churn = random.choices([0, 1], weights=[1 - prob_churn, prob_churn])[0]




    registro = {
        'antiguedad': antiguedad,
        'plan': plan,
        'metodo_pago': metodo_pago,
        'facturas_impagas': facturas_impagas,
        'frecuencia_uso': frecuencia_uso,
        'tickets_soporte': tickets_soporte,
        'tipo_contrato': tipo_contrato,
        'cambios_plan': cambios_plan,
        'canal_adquisicion': canal_adquisicion,
        'churn': churn
    }

    data.append(registro)

# Crear DataFrame
df = pd.DataFrame(data)

# Guardar dataset limpio
df.to_csv('dataset_limpio.csv', index=False)


print(f" Shape: {df.shape}")
print(f"\n Columnas:\n{df.columns.tolist()}")
print(df['churn'].value_counts(normalize=True))

 Shape: (5000, 10)

 Columnas:
['antiguedad', 'plan', 'metodo_pago', 'facturas_impagas', 'frecuencia_uso', 'tickets_soporte', 'tipo_contrato', 'cambios_plan', 'canal_adquisicion', 'churn']
churn
0    0.753
1    0.247
Name: proportion, dtype: float64


# 3. Script para ensuciar Dataset

In [9]:
import pandas as pd
import numpy as np
import random


# CARGAR DATASET LIMPIO ORIGINAL


df_original = pd.read_csv('dataset_limpio.csv')


print(" DATASET ORIGINAL")

print(f"Shape: {df_original.shape}")
print(f"Nulos: {df_original.isnull().sum().sum()}")


# FUNCIÓN PARA ENSUCIAR


def ensuciar_dataset(df, nivel='medio'):
    """
    Introduce valores nulos, outliers y ruido de forma controlada.
    """

    df_sucio = df.copy()

    porcentajes = {
        'bajo': {'nulos': 0.05, 'outliers': 0.03, 'ruido_label': 0.01},
        'medio': {'nulos': 0.10, 'outliers': 0.05, 'ruido_label': 0.02},
        'alto': {'nulos': 0.20, 'outliers': 0.10, 'ruido_label': 0.05}
    }

    config = porcentajes[nivel]
    n = len(df_sucio)


    print(f" ENSUCIANDO DATASET (Nivel: {nivel.upper()})")


    # 1. VALORES NULOS

    print(f"\n Introduciendo {config['nulos']*100}% valores nulos...")
    columnas_nulos = ['metodo_pago', 'facturas_impagas', 'frecuencia_uso', 'tickets_soporte', 'canal_adquisicion']

    for col in columnas_nulos:
        n_nulos = int(n * config['nulos'])
        indices_nulos = np.random.choice(df_sucio.index, size=n_nulos, replace=False)
        df_sucio.loc[indices_nulos, col] = np.nan
        print(f"   ↳ {col}: {n_nulos} nulos")

    # 2. OUTLIERS

    print(f"\n Introduciendo {config['outliers']*100}% outliers...")

    # Antiguedad: negativos o muy altos
    n_out = int(n * config['outliers'])
    indices = np.random.choice(df_sucio.index, size=n_out, replace=False)
    df_sucio.loc[indices, 'antiguedad'] = np.random.choice([-1, 0, 120, 150], size=n_out)
    print(f"    antiguedad: {n_out} outliers")

    # Frecuencia_uso: negativos o imposibles
    n_out = int(n * config['outliers'])
    indices = np.random.choice(df_sucio.index, size=n_out, replace=False)
    df_sucio.loc[indices, 'frecuencia_uso'] = np.random.choice([-5, -1, 50, 100], size=n_out)
    print(f"    frecuencia_uso: {n_out} outliers")

    # Tickets_soporte: muy altos
    n_out = int(n * config['outliers'])
    indices = np.random.choice(df_sucio.index, size=n_out, replace=False)
    df_sucio.loc[indices, 'tickets_soporte'] = np.random.randint(20, 100, size=n_out)
    print(f"    tickets_soporte: {n_out} outliers")

    # Facturas_impagas: negativos o muy altos
    n_out = int(n * config['outliers'] / 2)
    indices = np.random.choice(df_sucio.index, size=n_out, replace=False)
    df_sucio.loc[indices, 'facturas_impagas'] = np.random.choice([-1, 15, 20], size=n_out)
    print(f"    facturas_impagas: {n_out} outliers")

    # Cambios_plan: negativos o muy altos
    n_out = int(n * config['outliers'] / 2)
    indices = np.random.choice(df_sucio.index, size=n_out, replace=False)
    df_sucio.loc[indices, 'cambios_plan'] = np.random.choice([-1, 10, 15], size=n_out)
    print(f"    cambios_plan: {n_out} outliers")


    # 3. RUIDO EN LABELS (CRÍTICO)

    print(f"\n Introduciendo {config['ruido_label']*100}% ruido en etiquetas...")
    n_ruido = int(n * config['ruido_label'])
    indices = np.random.choice(df_sucio.index, size=n_ruido, replace=False)
    df_sucio.loc[indices, 'churn'] = 1 - df_sucio.loc[indices, 'churn']
    print(f"    churn: {n_ruido} etiquetas invertidas (0↔1)")


    # 4. INCONSISTENCIAS LÓGICAS

    print(f"\n Introduciendo inconsistencias lógicas...")

    # Alto uso + muchas facturas impagas
    mask = df_sucio['frecuencia_uso'] > 25
    if mask.sum() > 0:
        sample = df_sucio[mask].sample(n=min(30, mask.sum()))
        df_sucio.loc[sample.index, 'facturas_impagas'] = 3
        print(f"    {len(sample)} registros: alto uso + deudas")

    # Plan premium + muchos tickets
    mask = df_sucio['plan'] == 'premium'
    if mask.sum() > 0:
        sample = df_sucio[mask].sample(n=min(20, mask.sum()))
        df_sucio.loc[sample.index, 'tickets_soporte'] = np.random.randint(8, 15, size=len(sample))
        print(f"    {len(sample)} registros: premium + muchos tickets")

    # Cliente nuevo + contrato anual
    mask = df_sucio['antiguedad'] < 3
    if mask.sum() > 0:
        sample = df_sucio[mask].sample(n=min(15, mask.sum()))
        df_sucio.loc[sample.index, 'tipo_contrato'] = 'anual'
        print(f"    {len(sample)} registros: cliente nuevo + anual")


    # 5. VALORES CATEGÓRICOS INVÁLIDOS

    print(f"\n Introduciendo valores categóricos inválidos...")

    n_inv = int(n * 0.01)

    indices = np.random.choice(df_sucio.index, size=n_inv, replace=False)
    df_sucio.loc[indices, 'plan'] = np.random.choice(['Premium', 'BASICO', 'standard', 'vip'], size=n_inv)
    print(f"    plan: {n_inv} valores incorrectos")

    indices = np.random.choice(df_sucio.index, size=n_inv, replace=False)
    df_sucio.loc[indices, 'metodo_pago'] = np.random.choice(['Tarjeta Credito', 'EFECTIVO', 'paypal'], size=n_inv)
    print(f"    metodo_pago: {n_inv} valores incorrectos")


    # RESUMEN

    total_nulos = df_sucio.isnull().sum().sum()


    print(f" DATASET ENSUCIADO")

    print(f"Total valores nulos: {total_nulos}")
    print(f"Porcentaje nulos: {(total_nulos / (n * len(df_sucio.columns)))*100:.2f}%")
    print(f"Etiquetas con ruido: {n_ruido} ({(n_ruido/n)*100:.2f}%)")

    return df_sucio


# EJECUTAR


df_sucio = ensuciar_dataset(df_original, nivel='medio')
df_sucio.to_csv('dataset_churn_sucio.csv', index=False)

print("\n Dataset guardado: dataset_churn_sucio.csv")
print(f"\nShape: {df_sucio.shape}")
print(f"Churn distribution:\n{df_sucio['churn'].value_counts(normalize=True)}")

 DATASET ORIGINAL
Shape: (5000, 10)
Nulos: 0
 ENSUCIANDO DATASET (Nivel: MEDIO)

 Introduciendo 10.0% valores nulos...
   ↳ metodo_pago: 500 nulos
   ↳ facturas_impagas: 500 nulos
   ↳ frecuencia_uso: 500 nulos
   ↳ tickets_soporte: 500 nulos
   ↳ canal_adquisicion: 500 nulos

 Introduciendo 5.0% outliers...
    antiguedad: 250 outliers
    frecuencia_uso: 250 outliers
    tickets_soporte: 250 outliers
    facturas_impagas: 125 outliers
    cambios_plan: 125 outliers

 Introduciendo 2.0% ruido en etiquetas...
    churn: 100 etiquetas invertidas (0↔1)

 Introduciendo inconsistencias lógicas...
    30 registros: alto uso + deudas
    20 registros: premium + muchos tickets
    15 registros: cliente nuevo + anual

 Introduciendo valores categóricos inválidos...
    plan: 50 valores incorrectos
    metodo_pago: 50 valores incorrectos
 DATASET ENSUCIADO
Total valores nulos: 2439
Porcentaje nulos: 4.88%
Etiquetas con ruido: 100 (2.00%)

 Dataset guardado: dataset_churn_sucio.csv

Shape: (5000

In [10]:
df_sucio.head()

,antiguedad,plan,metodo_pago,facturas_impagas,frecuencia_uso,tickets_soporte,tipo_contrato,cambios_plan,canal_adquisicion,churn
0,16,estandar,transferencia_bancaria,2.0,NaN,0.0,mensual,3,call_center,0
1,120,estandar,efectivo,3.0,2.0,10.0,mensual,2,referido,1
2,29,premium,tarjeta_credito,NaN,28.0,1.0,anual,3,web,0
3,48,estandar,tarjeta_debito,0.0,9.0,2.0,mensual,1,referido,0
4,13,estandar,tarjeta_credito,1.0,19.0,5.0,mensual,0,web,0


# 4. Evaluo % de Churn en Dataset - Variable Target


In [5]:
tabla_proporciones = df_sucio['churn'].value_counts(normalize=True).to_frame(name='proportion')
print(tabla_proporciones)

       proportion
churn            
0          0.7478
1          0.2522


# 5. Guardo el Dataset generado


In [11]:
df_sucio.to_csv('dataset_sucio_final.csv', index=False)

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
